In [7]:
# -*- coding: utf-8 -*-
"""data_cleaning_3.ipynb (Enhanced Version)"""

import pandas as pd
import numpy as np
import re

# ==============================================================================
# 1. ĐỌC DỮ LIỆU & ĐỔI TÊN CỘT
# ==============================================================================
try:
    # Đường dẫn file gốc của bạn
    df = pd.read_csv('../data/raw/VN_housing_dataset.csv')
    print("✅ Đã đọc file thành công!")
    print(f"Kích thước ban đầu: {df.shape}")
except FileNotFoundError:
    print("❌ Lỗi: Không tìm thấy file csv!")
    exit()

# CẬP NHẬT: Không xóa vội 'Dài', 'Rộng' vì chúng ta cần dùng nó
cols_to_drop = ['Unnamed: 0', 'Ngày', 'Địa chỉ'] # Chỉ xóa cột không dùng được
df = df.drop(columns=cols_to_drop, errors='ignore')

# Đổi tên cột chuẩn (Thêm Length, Width)
rename_map = {
    'Quận': 'District',
    'Huyện': 'Ward',
    'Loại hình nhà ở': 'House_type',
    'Giấy tờ pháp lý': 'Legal',
    'Số tầng': 'Floors',
    'Số phòng ngủ': 'Bedrooms',
    'Diện tích': 'Area',
    'Giá/m2': 'Price_per_m2',
    'Dài': 'Length',   # Giữ lại
    'Rộng': 'Width',   # Giữ lại
}
df = df.rename(columns=rename_map)
print("✅ Đã đổi tên cột chuẩn.")

# ==============================================================================
# 2. XỬ LÝ SỐ LIỆU (DATA CLEANING)
# ==============================================================================
def extract_number(value):
    if pd.isna(value): return np.nan
    text = str(value).lower().replace(',', '.')
    match = re.search(r"[-+]?\d*\.\d+|\d+", text)
    return float(match.group()) if match else np.nan

# Áp dụng cho các cột số (bao gồm cả Dài, Rộng)
cols_num = ['Area', 'Price_per_m2', 'Bedrooms', 'Floors', 'Length', 'Width']
for col in cols_num:
    if col in df.columns:
        df[col] = df[col].apply(extract_number)

print("✅ Đã chuyển đổi dữ liệu sang dạng số.")

# ------------------------------------------------------------------------------
# 🌟 LOGIC MỚI (NÂNG CẤP): CỨU DỮ LIỆU DÀI / RỘNG THEO TỶ LỆ HÌNH HỌC
# ------------------------------------------------------------------------------
print(f"NaN trước khi xử lý: Dài={df['Length'].isna().sum()}, Rộng={df['Width'].isna().sum()}")

# Bước 1: Tính tỷ lệ Dài/Rộng đặc trưng của thị trường (Median Ratio) dựa trên dữ liệu có sẵn
# Chỉ lấy các dòng có đủ Dài và Rộng để tính toán
valid_rows = df.dropna(subset=['Length', 'Width']).copy()
# Đảm bảo tỷ lệ luôn >= 1 (Dài luôn >= Rộng) để tính cho chuẩn
valid_rows['Ratio'] = valid_rows[['Length', 'Width']].max(axis=1) / valid_rows[['Length', 'Width']].min(axis=1)
median_ratio = valid_rows['Ratio'].median()

print(f"ℹ️ Tỷ lệ Dài/Rộng trung bình của dữ liệu là: {median_ratio:.2f} (Nhà thường dài gấp {median_ratio:.2f} lần rộng)")

# Bước 2: Xử lý trường hợp thiếu 1 trong 2 (Giữ nguyên logic cũ vì nó đúng)
# Có Diện tích + Dài -> Tính Rộng
mask_w = df['Width'].isna() & df['Length'].notna() & df['Area'].notna()
df.loc[mask_w, 'Width'] = df.loc[mask_w, 'Area'] / df.loc[mask_w, 'Length']

# Có Diện tích + Rộng -> Tính Dài
mask_l = df['Length'].isna() & df['Width'].notna() & df['Area'].notna()
df.loc[mask_l, 'Length'] = df.loc[mask_l, 'Area'] / df.loc[mask_l, 'Width']

# Bước 3: Xử lý trường hợp THIẾU CẢ HAI (Dùng Median Ratio để suy diễn)
# Logic: Diện tích = Rộng * (Rộng * Ratio) -> Rộng = Sqrt(Diện tích / Ratio)
mask_both = df['Length'].isna() & df['Width'].isna() & df['Area'].notna()

# Tính Rộng giả định
df.loc[mask_both, 'Width'] = np.sqrt(df.loc[mask_both, 'Area'] / median_ratio)
# Tính Dài giả định
df.loc[mask_both, 'Length'] = df.loc[mask_both, 'Area'] / df.loc[mask_both, 'Width']

# Bước 4: Sắp xếp lại Dài và Rộng (Quy ước Dài là cạnh lớn hơn)
# Điều này giúp model học nhất quán: Length luôn >= Width
df['Temp_L'] = df[['Length', 'Width']].max(axis=1)
df['Temp_W'] = df[['Length', 'Width']].min(axis=1)
df['Length'] = df['Temp_L']
df['Width'] = df['Temp_W']
df.drop(columns=['Temp_L', 'Temp_W'], inplace=True)

# Điền thiếu cho các cột khác
df['Bedrooms'] = df['Bedrooms'].fillna(df['Bedrooms'].median())
df['Floors'] = df['Floors'].fillna(df['Floors'].median())
df['Legal'] = df['Legal'].fillna('Dang_cap_nhat')

print(f"NaN sau khi xử lý: Dài={df['Length'].isna().sum()}, Rộng={df['Width'].isna().sum()}")


# ==============================================================================
# 3. TẠO BIẾN MỤC TIÊU & LỌC NHIỄU (FILTERING)
# ==============================================================================
# Tính tổng giá
df['Total_Price_Billion'] = (df['Price_per_m2'] * df['Area']) / 1000

# Xử lý text Quận/Huyện/Phường
df['District'] = df['District'].str.replace('Quận', '').str.replace('Huyện', '').str.strip()
df['Ward'] = df['Ward'].str.replace('Phường', '').str.replace('Xã', '').str.strip()
# Điền khuyết thiếu cho cột Pháp lý
df['Legal'] = df['Legal'].fillna('Dang_cap_nhat')

# Lọc dữ liệu nhiễu (Logic cũ của bạn + Bổ sung IQR Filter cho giá)
df = df.dropna(subset=['District', 'Ward']) # Xóa nếu không có địa chỉ

df = df[(df['Area'] >= 10) & (df['Area'] <= 500)]
df = df[(df['Total_Price_Billion'] >= 0.5) & (df['Total_Price_Billion'] <= 100)]

# Lọc logic phòng ở
df = df[~((df['Area'] < 40) & (df['Bedrooms'] >= 8))]
df = df[~((df['Floors'] < 2) & (df['Bedrooms'] >= 5) & (df['Area'] < 100))]

# Lọc theo IQR của đơn giá (Price_per_m2) -> Giúp loại bỏ nhà giá ảo (quá rẻ/quá đắt)
Q1 = df['Price_per_m2'].quantile(0.10)
Q3 = df['Price_per_m2'].quantile(0.90)
df = df[(df['Price_per_m2'] >= Q1) & (df['Price_per_m2'] <= Q3)]

print(f"✅ Dữ liệu sạch cuối cùng: {len(df)} dòng")

# ==============================================================================
# [MỚI] LÀM TRÒN SỐ LIỆU (ROUNDING)
# ==============================================================================
# Làm tròn 2 chữ số thập phân cho Dài, Rộng và Tổng giá
cols_to_round = ['Length', 'Width', 'Total_Price_Billion', 'Area']
# Kiểm tra xem cột có tồn tại không trước khi làm tròn (tránh lỗi)
cols_exist = [c for c in cols_to_round if c in df.columns]

if cols_exist:
    df[cols_exist] = df[cols_exist].round(2)
    print(f"✅ Đã làm tròn 2 số thập phân cho các cột: {cols_exist}")

# Xem thử kết quả sau khi làm tròn
print("--- Dữ liệu sau khi làm tròn ---")
display(df[['Length', 'Width', 'Total_Price_Billion']].head())

# ==============================================================================
# 4. ONE-HOT ENCODING (MÃ HÓA)
# ==============================================================================
# Danh sách các cột cần mã hóa (Bao gồm cả Street mới)
categorical_cols = ['District', 'Ward', 'House_type', 'Legal']

# Tạo One-Hot
df_final = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Xóa các cột không cần thiết cho việc Train (Address, Price_per_m2...)
cols_garbage = ['Price_per_m2']
df_final = df_final.drop(columns=cols_garbage, errors='ignore')

print(f"Kích thước sau khi One-Hot: {df_final.shape}")

# ==============================================================================
# 5. LƯU FILE
# ==============================================================================
save_path = '../data/processed/clean_vn_housing.csv'
df_final.to_csv(save_path, index=False)
print(f"✅ Đã lưu file sạch tại: {save_path}")

✅ Đã đọc file thành công!
Kích thước ban đầu: (82497, 13)
✅ Đã đổi tên cột chuẩn.
✅ Đã chuyển đổi dữ liệu sang dạng số.
NaN trước khi xử lý: Dài=62670, Rộng=47052
ℹ️ Tỷ lệ Dài/Rộng trung bình của dữ liệu là: 2.50 (Nhà thường dài gấp 2.50 lần rộng)
NaN sau khi xử lý: Dài=1, Rộng=1
✅ Dữ liệu sạch cuối cùng: 65468 dòng
✅ Đã làm tròn 2 số thập phân cho các cột: ['Length', 'Width', 'Total_Price_Billion', 'Area']
--- Dữ liệu sau khi làm tròn ---


,Length,Width,Total_Price_Billion
0,10.72,4.29,4.0
1,9.62,3.85,4.3
2,10.00,4.00,2.6
3,12.75,4.00,5.1
4,9.00,4.00,3.1


Kích thước sau khi One-Hot: (65468, 246)
✅ Đã lưu file sạch tại: ../data/processed/clean_vn_housing.csv


In [8]:
df_final

,Floors,Bedrooms,Area,Length,Width,Total_Price_Billion,District_Bắc Từ Liêm,District_Chương Mỹ,District_Cầu Giấy,District_Gia Lâm,...,Ward_Đồng Xuân,Ward_Đội Cấn,Ward_Đức Giang,Ward_Đức Thắng,"House_type_Nhà mặt phố, mặt tiền","House_type_Nhà ngõ, hẻm",House_type_Nhà phố liền kề,Legal_Giấy tờ khác,Legal_Đang chờ sổ,Legal_Đã có sổ
0,4.0,5.0,46.0,10.72,4.29,4.0,False,False,True,False,...,False,False,False,False,False,True,False,False,False,True
1,5.0,3.0,37.0,9.62,3.85,4.3,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2,4.0,4.0,40.0,10.00,4.00,2.6,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
3,5.0,6.0,51.0,12.75,4.00,5.1,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
4,5.0,4.0,36.0,9.00,4.00,3.1,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82489,5.0,3.0,40.0,10.00,4.00,3.1,False,False,True,False,...,False,False,False,False,False,True,False,False,False,True
82490,5.0,3.0,38.0,9.75,3.90,2.9,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
82491,5.0,3.0,38.0,9.75,3.90,3.1,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
82494,5.0,4.0,60.0,12.25,4.90,6.1,False,False,True,False,...,False,False,False,False,False,True,False,False,False,True
